In [3]:
import pandas as pd
import string
from utils import *

In [4]:
import json
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [5]:
df = pd.read_json("data/natural_question/v1.0-simplified_simplified-nq-train.jsonl", lines=True, nrows=100)

In [6]:
df.head()

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [7]:
df.iloc[3]['question_text']

'who had the most wins in the nfl'

In [8]:
df.iloc[3]['document_text'].split(" ")[512:514]

['Tom', 'Brady']

In [9]:
dev_df = pd.read_json("data/natural_question/v1.0-simplified_nq-dev-all.jsonl", lines=True, nrows=2000)

In [10]:
dev_df.iloc[4]['question_text']

'star wars the clone wars anakin voice actor'

In [11]:
# dev_df.iloc[4]['annotations'] # 173-179

In [12]:
dev_df.iloc[150]['annotations'][1]


{'annotation_id': 5834450600727459786,
 'long_answer': {'candidate_index': 22,
  'end_byte': 197000,
  'end_token': 8433,
  'start_byte': 16013,
  'start_token': 684},
 'short_answers': [{'end_byte': 133179,
   'end_token': 5612,
   'start_byte': 133173,
   'start_token': 5611}],
 'yes_no_answer': 'NONE'}

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")
t5_tok = AutoTokenizer.from_pretrained("google/t5-large-ssm-nq")


In [14]:
input_ids = t5_tok("When was Franklin D. Roosevelt born?", return_tensors="pt").input_ids
gen_output = t5_qa_model.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

December 26, 1892


In [15]:
input_ids = t5_tok("Is 2+2=4?", return_tensors="pt").input_ids
gen_output = t5_qa_model.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

4


In [16]:
dev_df.iloc[4]['question_text']

# ' '.join([token['token'] for token in dev_df.iloc[100]['document_tokens'][1110:1112]])

'star wars the clone wars anakin voice actor'

In [17]:
normalize_answer('test')

'test'

In [18]:
tracker = []

def has_short_answer(annotations: list):
    if annotations:
        return sum([len(annotation['short_answers']) > 0 for annotation in annotations]) > 1
    else:
        False

def short_answer_string(sa_annotation, example):
    start_token = sa_annotation['start_token']
    end_token = sa_annotation['end_token']
    token_dicts = example['document_tokens'][start_token:end_token]
    return ' '.join([token_dict['token'] for token_dict in token_dicts])
        
def score_short_answer(prediction: str, example: dict):
    annotations = example['annotations']
    
    tracker_sa = []
    for annotation in annotations:
        short_answers = annotation['short_answers']
    
        exact_match = any([normalize_answer(short_answer_string(answer, example)) == normalize_answer(prediction) for answer in short_answers])
        tracker_sa.append([normalize_answer(short_answer_string(answer, example)) for answer in short_answers])
            
        if exact_match:
            break
    
    tracker.append({
        "annotated_answers": tracker_sa,
        "prediction": prediction,
        "exact_match": exact_match,
        "question": example["question_text"],
        "document_url": example["document_url"]
       })
    
    if exact_match:
        return 1
    else:
        return 0

score_short_answer('T.j. miller', dev_df.iloc[100])
    
    

1

In [19]:
def predict_short_answer(example):    
    input_ids = t5_tok(example['question_text'], return_tensors="pt").input_ids
    gen_output = t5_qa_model.generate(input_ids)[0]
    prediction = t5_tok.decode(gen_output, skip_special_tokens=True)
        
    return score_short_answer(prediction, example)

predict_short_answer(dev_df.iloc[100])
    

1

In [20]:
def score_short_answers(examples, nrows):
    
    correct, total, num_skipped = 0, 0, 0
    
    for i in range(nrows):
        if i % 30 == 0:
            print(i)
        
        example = examples.iloc[i]
        if not has_short_answer(example['annotations']):
            num_skipped += 1
            continue
        
        result = predict_short_answer(example)
        total += 1
        correct += result
    try:
        accuracy = round(correct/total, 4)
    except Exception:
        accuracy = None
    
    print(f"total: {total} correct: {correct} EM: {accuracy}, num_skipped: {num_skipped}")
            

In [21]:
tracker = []
score_short_answers(dev_df, 2000)



0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
total: 834 correct: 241 EM: 0.289, num_skipped: 1166


In [22]:
df = pd.DataFrame(tracker)

df.to_csv("results.csv", index=False)


In [28]:
test = tracker[0:100]

In [30]:
tracker[2]

{'annotated_answers': [['coldplay with special guest performers beyoncé and bruno mars'],
  ['coldplay', 'beyoncé', 'bruno mars'],
  [],
  ['british rock group coldplay with special guest performers beyoncé and bruno mars'],
  ['coldplay', 'beyoncé', 'bruno mars']],
 'prediction': 'Justin Timberlake',
 'exact_match': False,
 'question': 'who is playing the halftime show at super bowl 2016',
 'document_url': 'https://en.wikipedia.org//w/index.php?title=Super_Bowl_50_halftime_show&amp;oldid=823813276'}

In [44]:
refs = []
cands = []
for example in test:
    ref = []
    for answer in example['annotated_answers']:
        if answer:
            ref.append(answer[0])
    refs.append(ref)
    cands.append(normalize_answer(example['prediction']))
    

In [46]:
from bert_score import score

results = score(cands, refs, lang='en', verbose=True, rescale_with_baseline=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 37.81 seconds, 8.33 sentences/sec


In [48]:
results

(tensor([ 0.1212,  1.0000, -0.1675,  0.6930,  1.0000,  0.2203, -0.0268,  1.0000,
          0.0264,  0.9282,  0.1080,  1.0000,  0.8889,  1.0000,  0.2395,  0.6471,
          0.4463,  0.0262,  0.7499, -0.0270, -0.2745,  0.1586, -0.1735,  0.3625,
          0.1752, -0.1914,  1.0000,  1.0000,  0.5120,  0.1038,  0.7234,  0.1936,
          1.0000,  0.3189,  0.5127,  0.2742,  0.3217,  0.2758, -0.5421,  0.4419,
          1.0000,  0.6730,  1.0000,  0.8005,  1.0000, -0.0690, -0.1159,  0.1562,
          1.0000,  0.2088, -0.0939,  1.0000,  0.3590,  0.1684,  1.0000,  1.0000,
         -0.0313, -0.0774, -0.4675,  0.7390,  0.2593,  0.1652,  0.5723,  0.1990,
          0.3517,  0.2192,  1.0000,  0.2936,  1.0000,  0.2342, -0.0060,  0.1758,
          1.0000,  0.5013,  0.4530,  0.0358,  0.7662,  1.0000,  0.1979,  0.2929,
          0.3777,  1.0000,  1.0000,  0.7735,  0.2589, -0.1773,  0.4575,  0.3450,
          0.2314,  0.1356,  0.0528,  0.9031,  0.2687,  0.1796,  0.6415,  0.0408,
          1.0000, -0.0110,  

In [49]:
P, R, F1 = results

In [59]:
sum(F1) / len(F1)

tensor(0.3993)

In [56]:
refs_fake = []
cands_fake = []
for i in range(len(test)):
    example = test[i]
    prev_example = test[i-1]
    ref = []
    for answer in example['annotated_answers']:
        if answer:
            ref.append(answer[0])
    refs_fake.append(ref)
    cands_fake.append(normalize_answer(prev_example['prediction']))
    

In [57]:
fake_results = score(cands_fake, refs_fake, lang='en', verbose=True, rescale_with_baseline=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 37.75 seconds, 8.34 sentences/sec


In [62]:
sum(fake_results[0]) / len(fake_results[0])

tensor(-0.0216)